This notebook is about the sentiment analysis using Tensorflow and embedding layer.. Do upvote

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [ ]:
df=pd.read_csv('../input/imdb-dataset-sentiment-analysis-in-csv-format/Train.csv')
df.head()

In [ ]:
#shuffling data
df = df.sample(frac=1, axis=1).reset_index(drop=True)

check dataset is balanced or not

In [ ]:
df.groupby('label').count()

In [ ]:
import seaborn as sns
sns.countplot(df['label'])

In [ ]:

x_train,x_test,y_train,y_test= train_test_split(df['text'],df['label'],test_size=0.2)

In [ ]:
vocab_size=600
oov_tok='<OOV>'

In [ ]:
token=Tokenizer(num_words=vocab_size,oov_token=oov_tok)
token.fit_on_texts(x_train)

In [ ]:
wordIndex=token.word_index
sequences=token.texts_to_sequences(x_train)
train_padded_seq=pad_sequences(sequences,maxlen=60,padding='post',truncating='post')

In [ ]:
#testing data
test_sequences=token.texts_to_sequences(x_test)
test_padded_seq=pad_sequences(test_sequences,maxlen=60,padding='post',truncating='post')

In [ ]:
#model
model=tf.keras.models.Sequential([tf.keras.layers.Embedding(vocab_size,16,input_length=60),
                                  tf.keras.layers.GlobalAveragePooling1D(),
                                  tf.keras.layers.Dense(32,activation='relu'),
                                  tf.keras.layers.Dropout(0.4),
                                  tf.keras.layers.Dense(64,activation='relu'),
                                  tf.keras.layers.Dropout(0.3),
                                  tf.keras.layers.Dense(1,activation='sigmoid')
                                  
                                  
    
])

In [ ]:
model.summary()

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),metrics=['accuracy'],optimizer='adam')

In [ ]:
earlyStop=tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=2)
history=model.fit(train_padded_seq,y_train,validation_data=(test_padded_seq,y_test),epochs=10
                  ,callbacks=[earlyStop])

In [ ]:
model.evaluate(test_padded_seq, y_test)